# Langchain Multi Agent Implementation

## Introduction

Based on the user input, a **Router invokes an Agent** (langchain construct) from the pool of available agents. Each agent will decide and invoke the required **tool(s)** to get the response back user. This is achived by a single call to router_chain.invoke()

**Add packages** 

pip install
langchain==0.2.5
langchain-aws==0.1.12
langchain-community==0.2.5
langchain-core==0.2.20

In [1]:
!pip install -U langchain==0.2.5 langchain-aws==0.1.12 langchain-community==0.2.5 langchain-core==0.2.20

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 13.3 MB/s eta 0:00:00
  Using cached packaging-24.1-py3-none-any.whl.metadata (3.2 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 85.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 115.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 371.7/371.7 kB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 29.5 MB/s eta 0:00:00
Using cached packaging-24.1-py3-none-any.whl (53 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.9/423.9 kB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 112.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 28.3 MB/s eta 0:00:00
  

In [2]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_aws.chat_models.bedrock import ChatBedrock
from langchain_aws import ChatBedrockConverse
# from langchain_openai import AzureChatOpenAI
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage
from langchain.agents import XMLAgent
from langchain_core.prompts import PromptTemplate
from langchain.schema.runnable import RunnableBranch
from langchain_core.runnables import Runnable, RunnablePassthrough
from langchain.agents.format_scratchpad.tools import (
    format_to_tool_messages,
)
from langchain.agents.output_parsers.tools import ToolsAgentOutputParser
from langchain_core.output_parsers import StrOutputParser
import os

In [3]:
# select the needed llm
# llm = ChatBedrock(model_id="anthropic.claude-3-5-sonnet-20240620-v1:0", region_name='us-east-1', beta_use_converse_api=True)
llm = ChatBedrock(model_id="anthropic.claude-3-sonnet-20240229-v1:0", region_name='us-east-1', beta_use_converse_api=True)
# llm = ChatBedrockConverse(model="anthropic.claude-3-sonnet-20240229-v1:0", temperature= 0)
# llm = ChatBedrockConverse(model="anthropic.claude-3-5-sonnet-20240620-v1:0", temperature= 0)
#llm = AzureChatOpenAI(temperature=0, model_name="gpt-4o-2024-05-13")

In [4]:
nsm_prompt = ChatPromptTemplate.from_messages(
            [
                SystemMessage(content="You are an agent to list down Online and Offline Firewalls"),
                MessagesPlaceholder(variable_name="chat_history", optional=True),
                HumanMessagePromptTemplate.from_template(
                    input_variables=["input"], template="{input}"
                ),
                MessagesPlaceholder(variable_name="agent_scratchpad")
            ]
        )

In [5]:
@tool
def online_firewalls():
    """This tool provides json of online firewalls"""
    return {"firewall1": "A", "firewall2": "B"}
@tool
def offline_firewalls():
    """This tool provides json of offline firewalls"""
    return [{"firewallA": 1}, {"firewallB": 2}]

nsm_tools = [online_firewalls, offline_firewalls]

In [6]:
nsm_agent_w_tools = create_tool_calling_agent(llm=llm, tools=nsm_tools, prompt = nsm_prompt)

In [7]:
nsm_agent_w_tools_executor = AgentExecutor(agent=nsm_agent_w_tools, tools=nsm_tools, verbose=True)

In [8]:
output = nsm_agent_w_tools_executor.invoke({"input": "Which firewalls are online"})
response = output['output'][0]['text']
print(response)



> Entering new AgentExecutor chain...

Invoking: `online_firewalls` with `{}`
responded: [{'type': 'text', 'text': 'To get the list of online firewalls, I will invoke the "online_firewalls" tool:', 'index': 0}, {'type': 'tool_use', 'name': 'online_firewalls', 'id': 'tooluse_9uSidlckSgKkXTcgZ233Gw', 'index': 1, 'input': ''}]

{'firewall1': 'A', 'firewall2': 'B'}[{'type': 'text', 'text': '\n\nThe online firewalls are: firewall1 and firewall2.', 'index': 0}]

> Finished chain.


The online firewalls are: firewall1 and firewall2.


In [9]:
msw_prompt = ChatPromptTemplate.from_messages(
            [
                SystemMessage(content="You are an agent to list all available Firewalls"),
                MessagesPlaceholder(variable_name="chat_history", optional=True),
                HumanMessagePromptTemplate.from_template(
                    input_variables=["input"], template="{input}"
                ),
                MessagesPlaceholder(variable_name="agent_scratchpad")
            ]
        )

In [10]:
@tool
def all_firewalls():
    """This tool provides json of all firewalls"""
    all_firewalls = {"firewall1": "A", "firewall2": "B", "firewall3": "C", "firewall4": "D", "firewall5": "E", "firewall6": "F"}
    return all_firewalls

msw_tools = [all_firewalls]

In [11]:
msw_agent_w_tools = create_tool_calling_agent(llm=llm, tools=msw_tools, prompt = msw_prompt)

In [12]:
msw_agent_w_tools_executor = AgentExecutor(agent=msw_agent_w_tools, tools=msw_tools, verbose=True)

In [13]:
output = msw_agent_w_tools_executor.invoke({"input": "what are all the firewalls I have?"})
response = output['output'][0]['text']
print(response)



> Entering new AgentExecutor chain...

Invoking: `all_firewalls` with `{}`
responded: [{'type': 'text', 'text': 'To get a list of all available firewalls, we can invoke the "all_firewalls" tool:', 'index': 0}, {'type': 'tool_use', 'name': 'all_firewalls', 'id': 'tooluse_w65XKSkUT_mwRrinjae0qQ', 'index': 1, 'input': ''}]

{'firewall1': 'A', 'firewall2': 'B', 'firewall3': 'C', 'firewall4': 'D', 'firewall5': 'E', 'firewall6': 'F'}[{'type': 'text', 'text': '\n\nThis tool provides JSON output listing all the firewalls available to you. Based on the results, the firewalls you have are:\n\n- firewall1\n- firewall2  \n- firewall3\n- firewall4\n- firewall5\n- firewall6', 'index': 0}]

> Finished chain.


This tool provides JSON output listing all the firewalls available to you. Based on the results, the firewalls you have are:

- firewall1
- firewall2  
- firewall3
- firewall4
- firewall5
- firewall6


In [14]:
from langchain_core.pydantic_v1 import BaseModel, Field

class RouterUserInput(BaseModel):
    query: str = Field(description="should be the entire input from the user")

In [15]:
@tool(args_schema=RouterUserInput)
def nsm_agent(query: str) -> str:
    '''this is NSM Agent that returns list of firewalls that are online and offline'''
    nsm_agent_output = nsm_agent_w_tools_executor.invoke({"input": query})
    return nsm_agent_output['output'][0]['text']

In [16]:
@tool(args_schema=RouterUserInput)
def msw_agent(query: str) -> str:
    '''this is MSW Agent that returns inventory of firewalls'''
    msw_agent_output = msw_agent_w_tools_executor.invoke({"input": query})
    return msw_agent_output['output'][0]['text']

In [17]:
router_tools= [nsm_agent, msw_agent]

In [18]:
router_prompt = ChatPromptTemplate.from_messages(
            [
                SystemMessage(content="You are an agent to help with firewall queries, make sure to return exactly what was requested in your response"),
                MessagesPlaceholder(variable_name="chat_history", optional=True),
                HumanMessagePromptTemplate.from_template(
                    input_variables=["input"], template="{input}"
                ),
                MessagesPlaceholder(variable_name="agent_scratchpad")
            ]
        )

router_agent = create_tool_calling_agent(llm=llm, tools=router_tools, prompt = router_prompt)

In [19]:
router_agent_executor = AgentExecutor(agent=router_agent, tools=router_tools, verbose=True)

In [20]:
router_agent_executor.invoke({"input":"Which firewalls are offline"})



> Entering new AgentExecutor chain...

Invoking: `nsm_agent` with `{'query': 'Which firewalls are offline'}`
responded: [{'type': 'tool_use', 'name': 'nsm_agent', 'id': 'tooluse_6INiVlSSTOW_Fm-u6N5T_w', 'index': 0, 'input': '{"query": "Which firewalls are offline"}'}]



> Entering new AgentExecutor chain...

Invoking: `offline_firewalls` with `{}`
responded: [{'type': 'text', 'text': 'To get the list of offline firewalls, I will invoke the "offline_firewalls" tool:', 'index': 0}, {'type': 'tool_use', 'name': 'offline_firewalls', 'id': 'tooluse_ihcqm5GuQAubjl6w8M7jLA', 'index': 1, 'input': ''}]

[{'firewallA': 1}, {'firewallB': 2}][{'type': 'text', 'text': '\n\nThe offline firewalls according to the tool output are firewallA and firewallB.', 'index': 0}]

> Finished chain.


The offline firewalls according to the tool output are firewallA and firewallB.[]

> Finished chain.


{'input': 'Which firewalls are offline', 'output': []}

In [21]:
router_agent_executor.invoke({"input":"Which firewalls are online"})



> Entering new AgentExecutor chain...

Invoking: `nsm_agent` with `{'query': 'Which firewalls are online'}`
responded: [{'type': 'text', 'text': 'Here is how we can find out which firewalls are online:', 'index': 0}, {'type': 'tool_use', 'name': 'nsm_agent', 'id': 'tooluse_vRctYupUTVuiCAXV0LtIGw', 'index': 1, 'input': '{"query": "Which firewalls are online"}'}]



> Entering new AgentExecutor chain...

Invoking: `online_firewalls` with `{}`
responded: [{'type': 'text', 'text': 'To get the list of online firewalls, I will invoke the "online_firewalls" tool:', 'index': 0}, {'type': 'tool_use', 'name': 'online_firewalls', 'id': 'tooluse_h_OT3cKtQZOXGzzr50id3A', 'index': 1, 'input': ''}]

{'firewall1': 'A', 'firewall2': 'B'}[{'type': 'text', 'text': '\n\nThe online firewalls based on the tool output are: firewall1 and firewall2.', 'index': 0}]

> Finished chain.


The online firewalls based on the tool output are: firewall1 and firewall2.[{'type': 'text', 'text': '\n\nThe nsm_agent tool 

{'input': 'Which firewalls are online',
 'output': [{'type': 'text',
   'text': '\n\nThe nsm_agent tool lists the firewalls that are currently online and offline, so invoking it with the query "Which firewalls are online" allows me to extract just the online firewall names from its output.',
   'index': 0}]}

In [22]:
router_agent_executor.invoke({"input":"what are all the firewalls I have?"})



> Entering new AgentExecutor chain...

Invoking: `msw_agent` with `{'query': 'what are all the firewalls I have?'}`
responded: [{'type': 'text', 'text': 'To get the inventory of all firewalls, we can invoke the msw_agent tool:', 'index': 0}, {'type': 'tool_use', 'name': 'msw_agent', 'id': 'tooluse_sEEICtIiRHqfHUj9CkPzAw', 'index': 1, 'input': '{"query": "what are all the firewalls I have?"}'}]



> Entering new AgentExecutor chain...

Invoking: `all_firewalls` with `{}`
responded: [{'type': 'text', 'text': 'Okay, let me invoke the "all_firewalls" tool to get the list of available firewalls:', 'index': 0}, {'type': 'tool_use', 'name': 'all_firewalls', 'id': 'tooluse_ZXBcjkPAQsyDq2w4-HQ42w', 'index': 1, 'input': ''}]

{'firewall1': 'A', 'firewall2': 'B', 'firewall3': 'C', 'firewall4': 'D', 'firewall5': 'E', 'firewall6': 'F'}[{'type': 'text', 'text': '\n\nThe tool returned a JSON object listing all the available firewalls. Based on the output, you have the following firewalls:\n\n- fire

{'input': 'what are all the firewalls I have?', 'output': []}

In [23]:
router_agent_executor.invoke({"input":"what are all the firewalls I have and are they online?"})



> Entering new AgentExecutor chain...

Invoking: `nsm_agent` with `{'query': 'what are all the firewalls I have and are they online?'}`
responded: [{'type': 'text', 'text': 'To get the list of all firewalls and their status, we can invoke the nsm_agent tool:', 'index': 0}, {'type': 'tool_use', 'name': 'nsm_agent', 'id': 'tooluse_1VGQJL9bTzGuCjfmGi5TJQ', 'index': 1, 'input': '{"query": "what are all the firewalls I have and are they online?"}'}]



> Entering new AgentExecutor chain...

Invoking: `online_firewalls` with `{}`
responded: [{'type': 'text', 'text': 'To get the list of online and offline firewalls, I will invoke the relevant tools:', 'index': 0}, {'type': 'tool_use', 'name': 'online_firewalls', 'id': 'tooluse_e7nZP1RMR4SD2YVlqz34Mw', 'index': 1, 'input': ''}]

{'firewall1': 'A', 'firewall2': 'B'}
Invoking: `offline_firewalls` with `{}`
responded: [{'type': 'text', 'text': '\n\nThe online firewalls are firewall1 and firewall2.', 'index': 0}, {'type': 'tool_use', 'name': 'of

{'input': 'what are all the firewalls I have and are they online?',
 'output': [{'type': 'text',
   'text': '\n\nThe nsm_agent returned the list of all your firewalls, indicating which ones are online and which are offline. firewall1 and firewall2 are online, while firewallA and firewallB are currently offline.',
   'index': 0}]}